In [1]:
import aocd
import numpy as np
import math
from collections import defaultdict
from itertools import product, permutations
%run helper.ipynb
puzzle = aocd.models.Puzzle(year=2021, day=19)
data = puzzle.input_data.split("\n\n")
scanned_points = [[tuple(map(int, y.split(","))) for y in x.split("\n")[1:]] for x in data]

In [2]:
def all_dist(points):
    point_map = {}
    for source in points:
        point_map[source] = set([source.s_dist(dest) for dest in points if source != dest])
    return point_map

In [3]:
def get_rot_matrix(vec1, vec2):
    if(len(set(map(abs, vec1))) != 3):
        return None
    abs1 = tuple(map(abs, vec1))
    abs2 = tuple(map(abs, vec2))
    for s in permutations((0,1,2)):
        if(tuple([abs2[s[x]] for x in range(3)]) == abs1):
            shift = s
            break
    else:
        return None
    shifted = [vec2[shift[x]] for x in range(3)]
    polarity = [shifted[x]/vec1[x] for x in range(3)]
    
    r = np.zeros((3,3), int)
    for i in range(3):
        r[i][shift[i]] = polarity[i]
    
    return r

In [4]:
class Scanner():
    def __init__(self, number, beacons):
        self.beacons = [Point(x) for x in beacons]  # list of tuples
        self.beacon_dist_map = all_dist(self.beacons)  # dict of above tuples to distance sets
        self.number = number
        self.oriented = False
        self.position = Point((0, 0, 0))
    
    def orient(self, other):
        if(other.oriented == True):
            return True
        matched_points = []
        possible_pairs = product(self.beacon_dist_map.keys(), other.beacon_dist_map.keys())
        for (ka, kb) in possible_pairs:
            if(len(self.beacon_dist_map[ka] & other.beacon_dist_map[kb]) >= 10):
                matched_points.append((ka, kb))
        if(len(matched_points) >= 12):
            d1 = np.asarray(matched_points[0][0] - matched_points[1][0])
            d2 = np.asarray(matched_points[0][1] - matched_points[1][1])
            r = get_rot_matrix(d1, d2)
            shift = matched_points[0][0] - Point(np.matmul(r, matched_points[0][1]))

            other.beacons = [Point(np.matmul(r, x)) + shift for x in other.beacons]
            other.beacon_dist_map = all_dist(other.beacons)
            other.oriented = True
            other.position = shift
            return True
        return False

In [5]:
s = [Scanner(x[0], x[1]) for x in enumerate(scanned_points)]
s[0].oriented = True

In [6]:
unoriented = [None]
while(len(unoriented) != 0):
    oriented = [x for x in s if x.oriented]
    unoriented = [x for x in s if not x.oriented]
    [o.orient(u) for (o, u) in product(oriented, unoriented)]

In [7]:
puzzle.answer_a = len(set(flatten([x.beacons for x in s])))

In [8]:
scan_pairs = product(s, s)
ls = []
for pair in scan_pairs:
    ls.append(pair[0].position.m_dist(pair[1].position))
puzzle.answer_b = str(max(ls))